# Testing OpenSmile Classes

Draft 1 of pipeline for creating a df, loading the generated openSMILE features, and running feature selection.

In [ ]:
import pandas as pd
import numpy as np
import sys
import os
import json
from tqdm import tqdm
sys.path.append("/home/ubuntu/MultiModalDeepFake")
from packages.LJDataLoader import LJDataLoader
from packages.SmileFeatureLoader import smileFeatureLoader
from packages.SmileFeatureSelector import smileFeatureSelectorBruteForce, smileFeatureSelectFromModel, smileUnivariateFeatureSelector, process_data_for_all_archs
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import sklearn.feature_selection as fs
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, confusion_matrix, classification_report

## Part 1: Creating the dataframe with the train-dev-test split

In [ ]:
#load LJ 16khz data
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_Original.csv'
loader = LJDataLoader(data_path=file_path)

In [ ]:
#split data into train, dev, test
loader.splitData()

13057
7834 10445


In [ ]:
source_architectures = ['Full_Band_MelGan', 'HifiGan', 'MelGan', 'MelGanLarge', 'Multi_Band_MelGan', 'Parallel_WaveGan', 'Waveglow']
new_col_name = 'RandWaveFake'

In [ ]:
loader.selectRandomArchitecture(target_col=new_col_name, source_cols=source_architectures)

In [ ]:
loader.metadata.columns

Index(['index', 'Unnamed: 0', 'id', 'Main_ID', 'Secondary_ID', 'Real',
       'Full_Band_MelGan', 'HifiGan', 'MelGan', 'MelGanLarge',
       'Multi_Band_MelGan', 'Parallel_WaveGan', 'Waveglow', 'ElevenLabs',
       'UberDuck', 'transcript_1', 'transcript_2', 'type', 'RandWaveFake'],
      dtype='object')

In [ ]:
#generate the dataframe
#single_id entry=True breaks
df = loader.generateFinalDataFrame(real_col='Real', fake_cols=['ElevenLabs','UberDuck','RandWaveFake'])

In [ ]:
df.shape

(52228, 6)

In [ ]:
df.head()

,type,id,architecture,path,label,multiclass_label
0,train,LJ013-0125,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
1,train,LJ042-0084,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
2,train,LJ047-0032,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
3,train,LJ044-0063,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
4,train,LJ016-0067,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0


In [ ]:
df['architecture'].value_counts()

Real            13057
ElevenLabs      13057
UberDuck        13057
RandWaveFake    13057
Name: architecture, dtype: int64

#### Temporary solution for all_archs:

This external function could be integrated into data loader class

In [ ]:
fake_cols=['ElevenLabs','UberDuck','RandWaveFake']

arch_filter = process_data_for_all_archs(df, fake_cols)

In [ ]:
for key,value in arch_filter.items():
    print(key, len(value))

ElevenLabs 4352
UberDuck 4352
RandWaveFake 4353


In [ ]:
df2 = pd.DataFrame()
for key, value in arch_filter.items():
    df_ = df[(df['architecture'].isin(['Real', key])) & (df['id'].isin(value))].copy()
    df_.loc[:, 'arch_filter'] = [key] * df_.shape[0]
    df2 = pd.concat((df2, df_))

In [ ]:
df2.architecture.value_counts()

Real            13057
RandWaveFake     4353
ElevenLabs       4352
UberDuck         4352
Name: architecture, dtype: int64

In [ ]:
df2.arch_filter.value_counts()

RandWaveFake    8706
ElevenLabs      8704
UberDuck        8704
Name: arch_filter, dtype: int64

In [ ]:
merge_df = df2[['path','arch_filter']]

In [ ]:
merge_df.shape, df2.shape

((26114, 2), (26114, 7))

In [ ]:
final_df = pd.merge(df, merge_df, on='path', how='left')

In [ ]:
final_df.arch_filter.value_counts(dropna=False)

NaN             26114
RandWaveFake     8706
UberDuck         8704
ElevenLabs       8704
Name: arch_filter, dtype: int64

In [ ]:
df.shape, final_df.shape

((52228, 6), (52228, 7))

## Side: Creating/Updating File Mappings JSON:

This JSON has the path to the collection of wavfiles as the key and the corresponding smile features as values. 

For now, each time a new set of wav file are generated for a new provider/architecture - they should be stored in a separate unique folderpath. The `SmileFeatureGenerator` class will convert these to the csv file that can then be used to create a new JSON entry. 

In [ ]:
file_mapping = dict()

#biden real
file_mapping['/home/ubuntu/data/world-leaders-dataset/WavFiles/biden_wav_audio/44KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-world-leaders-dataset-WavFiles-biden_wav_audio-44KHz.csv'
file_mapping['/home/ubuntu/data/world-leaders-dataset/WavFiles/biden_wav_audio/16KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-world-leaders-dataset-WavFiles-biden_wav_audio-16KHz.csv'

#biden eleven labs
file_mapping['/home/ubuntu/data/world-leaders-dataset/11LabsDeepFakes/Biden/44Khz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-world-leaders-dataset-11LabsDeepFakes-Biden-44Khz.csv'
file_mapping['/home/ubuntu/data/world-leaders-dataset/11LabsDeepFakes/Biden/16Khz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-world-leaders-dataset-11LabsDeepFakes-Biden-16Khz.csv'

#biden in the wild
file_mapping['/home/ubuntu/data/InTheWildFakes/Biden/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-InTheWildFakes-Biden-Original.csv'
file_mapping['/home/ubuntu/data/InTheWildFakes/Biden/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-InTheWildFakes-Biden-16000KHz.csv'

#ljspeech real
file_mapping['/home/ubuntu/data/wavefake_data/LJSpeech_1.1/wavs/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-LJSpeech_1.1-wavs-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/LJSpeech_1.1/wavs/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-LJSpeech_1.1-wavs-16000KHz.csv'

#ljspeech eleven labs
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_elevenlabs/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_elevenlabs-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_elevenlabs/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_elevenlabs-16000KHz.csv'

#ljspeech uberduck
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_uberduck/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-06-wavefake_data-generated_audio-ljspeech_uberduck-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_uberduck/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-06-wavefake_data-generated_audio-ljspeech_uberduck-16000KHz.csv'

#ljspeech melgan
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_melgan/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_melgan-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_melgan/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_melgan-16000KHz.csv'

#ljspeech melgan large
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_melgan_large/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_melgan_large-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_melgan_large/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_melgan_large-16000KHz.csv'

#ljspeech multi band melgan
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_multi_band_melgan/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_multi_band_melgan-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_multi_band_melgan/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_multi_band_melgan-16000KHz.csv'

#ljspeech full band melgan
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_full_band_melgan/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_full_band_melgan-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_full_band_melgan/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_full_band_melgan-16000KHz.csv'

#ljspeech hifiGAN
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_hifiGAN/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_hifiGAN-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_hifiGAN/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_hifiGAN-16000KHz.csv'

#ljspeech parallel wavegan
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_parallel_wavegan/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_parallel_wavegan-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_parallel_wavegan/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_parallel_wavegan-16000KHz.csv'

#ljspeech waveglow
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_waveglow/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_waveglow-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_waveglow/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_waveglow-16000KHz.csv'

#file_mapping

In [ ]:
cellEnabled=0

if cellEnabled:

    file_mappings = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/' + 'file_mappings.json' 

    with open(file_mappings, "w") as json_file:
        json.dump(file_mapping, json_file, indent=4)

## Part 2: Loading the openSMILE features

The openSMILE features are generated and stored in multiple `csv` files here - `data/OpenSmile/04-2023-OSF-feature-csvs`. This folder also stores the JSON file

In [ ]:
final_df.shape

(52228, 7)

In [ ]:
#pass the test df and the smileFeatureLoader object does the merging
smFeatLoader = smileFeatureLoader(final_df)

100%|██████████| 10/10 [02:15<00:00, 13.54s/it]


In [ ]:
#get the df for feature selection
fs_df = smFeatLoader.return_merged_df()

In [ ]:
fs_df.shape

(52226, 6381)

In [ ]:
df.shape

(52228, 6)

In [ ]:
fs_df.head()

,type,id,architecture,path,label,multiclass_label,arch_filter,duration(seconds),audspec_lengthL1norm_sma_range,audspec_lengthL1norm_sma_maxPos,...,mfcc_sma_de[14]_peakRangeAbs,mfcc_sma_de[14]_peakRangeRel,mfcc_sma_de[14]_peakMeanAbs,mfcc_sma_de[14]_peakMeanMeanDist,mfcc_sma_de[14]_peakMeanRel,mfcc_sma_de[14]_minRangeRel,mfcc_sma_de[14]_meanRisingSlope,mfcc_sma_de[14]_stddevRisingSlope,mfcc_sma_de[14]_meanFallingSlope,mfcc_sma_de[14]_stddevFallingSlope
0,train,LJ013-0125,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,UberDuck,7.600,2.150976,0.856192,...,6.666750,0.561728,2.251509,2.245225,19.983110,0.502100,104.51715,42.642437,102.34870,44.233257
1,train,LJ042-0084,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,RandWaveFake,8.517,2.152849,0.801663,...,8.883474,0.466906,3.512975,3.513055,-20.000000,0.595703,138.78543,64.106000,135.07686,69.522630
2,train,LJ047-0032,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,ElevenLabs,8.203,1.993151,0.070284,...,9.228073,0.445313,3.162120,3.158883,19.999882,0.725314,130.25446,67.431730,124.56447,68.578705
3,train,LJ044-0063,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,RandWaveFake,3.025,3.101600,0.054422,...,7.837853,0.519312,2.595290,2.634245,-18.663334,0.602941,114.82919,67.769290,113.65942,59.367180
4,train,LJ016-0067,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,RandWaveFake,10.049,2.455078,0.045226,...,9.766602,0.581217,2.960240,2.954486,19.996292,0.609338,137.75078,60.657215,129.61658,60.697903


In [ ]:
#get the columns that are not the 6373 features as metadata
metadata = list(fs_df.columns[:7])
metadata

['type',
 'id',
 'architecture',
 'path',
 'label',
 'multiclass_label',
 'arch_filter']

## Part 3: Running Brute Force Feature Selection

In [ ]:
fs_df.shape

(52226, 6381)

In [ ]:
#initialize the object
smFS_bf = smileFeatureSelectorBruteForce(df=fs_df)

Initializing data...
Standardizing features...
smileFeatureSelectorBruteForce object initialized.



In [ ]:
smFS_bf.generate_data()

Generating brute force feature selection data...


100%|██████████| 6373/6373 [04:23<00:00, 24.14it/s]


Brute force feature selection data generated.
All data stored in self.bffs_data.



In [ ]:
smFS_bf.bffs_data.sort_values(by='ElevenLabs', ascending=False)

,features,ElevenLabs,UberDuck,RandWaveFake,average_binary,all_archs,multiclass
96,pcm_zcr_sma_quartile1,0.998468,0.600153,0.500574,0.699732,0.662007,0.548640
93,pcm_zcr_sma_range,0.995596,0.580429,0.513022,0.696349,0.653198,0.529395
103,pcm_zcr_sma_percentile99.0,0.995021,0.541172,0.518767,0.684987,0.641708,0.517714
104,pcm_zcr_sma_pctlrange0-1,0.994255,0.533321,0.521256,0.682944,0.640176,0.513118
227,pcm_zcr_sma_de_percentile99.0,0.993489,0.556683,0.519341,0.689838,0.622941,0.534374
...,...,...,...,...,...,...,...
2562,audSpec_Rfilt_sma_de[19]_upleveltime50,0.457488,0.513596,0.501340,0.490808,0.476446,0.234872
4862,pcm_fftMag_fband1000-4000_sma_qregc1,0.457296,0.548449,0.496936,0.500894,0.528342,0.251628
175,audspecRasta_lengthL1norm_sma_de_upleveltime50,0.454998,0.543278,0.504404,0.500894,0.497319,0.270203
2493,audSpec_Rfilt_sma_de[17]_skewness,0.453083,0.557641,0.500574,0.503766,0.496170,0.280065


In [ ]:
smFS_bf.FS_from_col(num_features=15, sort_col='ElevenLabs')


Top 15 features selected from ElevenLabs architecture.

1. pcm_zcr_sma_quartile1
2. pcm_zcr_sma_range
3. pcm_zcr_sma_percentile99.0
4. pcm_zcr_sma_pctlrange0-1
5. pcm_zcr_sma_de_percentile99.0
6. pcm_zcr_sma_de_pctlrange0-1
7. pcm_zcr_sma_quartile2
8. pcm_zcr_sma_de_range
9. pcm_zcr_sma_de_percentile1.0
10. pcm_zcr_sma_percentile1.0
11. pcm_fftMag_spectralSlope_sma_percentile99.0
12. audspec_lengthL1norm_sma_de_peakMeanRel
13. pcm_zcr_sma_stddev
14. pcm_zcr_sma_rqmean
15. pcm_zcr_sma_peakMeanAbs


In [ ]:
smFS_bf.FS_top_n_from_all_archs(n=15)


Top features selected from all architectures.

1. pcm_zcr_sma_quartile1
2. jitterLocal_sma_flatness
3. mfcc_sma[13]_lpgain
4. mfcc_sma_de[9]_meanRisingSlope
5. jitterDDP_sma_quartile2
6. pcm_zcr_sma_peakMeanAbs
7. pcm_zcr_sma_percentile99.0
8. pcm_zcr_sma_de_range
9. mfcc_sma_de[14]_lpgain
10. shimmerLocal_sma_quartile1
11. mfcc_sma[14]_quartile2
12. mfcc_sma[5]_lpc2
13. jitterDDP_sma_flatness
14. jitterDDP_sma_percentile1.0
15. pcm_zcr_sma_pctlrange0-1
16. shimmerLocal_sma_percentile1.0
17. pcm_zcr_sma_de_pctlrange0-1
18. pcm_fftMag_spectralSlope_sma_percentile99.0
19. mfcc_sma_de[11]_lpgain
20. mfcc_sma_de[14]_meanFallingSlope
21. mfcc_sma[14]_quartile3
22. jitterLocal_sma_quartile1
23. pcm_zcr_sma_stddev
24. mfcc_sma[14]_amean
25. mfcc_sma_de[13]_meanRisingSlope
26. jitterLocal_sma_percentile1.0
27. mfcc_sma_de[13]_lpgain
28. mfcc_sma[14]_peakMeanAbs
29. pcm_zcr_sma_quartile2
30. pcm_zcr_sma_percentile1.0
31. pcm_zcr_sma_rqmean
32. pcm_zcr_sma_range
33. mfcc_sma[11]_lpc2
34. audspe

#### Running multiclass prediction:

In [ ]:
features = smFS_bf.FS_top_n_from_all_archs(n=15, return_features=True, print_features=False)

In [ ]:
metadata=['type', 'id', 'architecture', 'path', 'label', 'multiclass_label', 'duration(seconds)']
ml_data = fs_df[fs_df.columns.intersection(metadata + features)]

In [ ]:
model = LogisticRegression(max_iter=1000000, n_jobs=-1, solver='saga')

In [ ]:
train_df = ml_data[ml_data['type'] == 'train'].copy()
test_df = ml_data[ml_data['type'] == 'test'].copy()

In [ ]:
X_train = train_df[features]
X_test = test_df[features]

y_train = train_df['multiclass_label']
y_test = test_df['multiclass_label']

model.fit(X_train, y_train)
y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)

tra_acc = accuracy_score(y_train, y_hat_train)
tst_acc = accuracy_score(y_test, y_hat_test)

tra_acc, tst_acc

(0.8895484282750917, 0.8893462237963051)

## Part 4: Running Select From Model

In [ ]:
smFS_model = smileFeatureSelectFromModel(fs_df)

Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.



In [ ]:
smFS_model.select_features_binary()


Selected features:.

1. mfcc_sma[13]_lpgain
2. mfcc_sma[8]_lpgain
3. jitterDDP_sma_quartile1
4. mfcc_sma[9]_lpgain
5. jitterDDP_sma_percentile1.0
6. pcm_zcr_sma_percentile99.0
7. mfcc_sma_de[13]_lpgain
8. mfcc_sma[12]_lpgain
9. mfcc_sma_de[9]_lpgain
10. mfcc_sma[12]_meanPeakDist


In [ ]:
smFS_model.select_features_binary(max_features=10, archs=['ElevenLabs', 'UberDuck'])


Selected features:.

1. mfcc_sma[5]_lpc2
2. mfcc_sma[13]_lpgain
3. audspec_lengthL1norm_sma_rqmean
4. mfcc_sma[9]_lpgain
5. pcm_zcr_sma_percentile99.0
6. mfcc_sma_de[13]_lpgain
7. mfcc_sma[14]_peakMeanAbs
8. pcm_zcr_sma_peakRangeAbs
9. mfcc_sma_de[9]_lpgain
10. mfcc_sma[14]_quartile2


In [ ]:
smFS_model.select_features_binary(max_features=10, archs='ElevenLabs')


Selected features:.

1. pcm_zcr_sma_quartile1
2. pcm_zcr_sma_stddev
3. pcm_zcr_sma_rqmean
4. voicingFinalUnclipped_sma_flatness
5. audspec_lengthL1norm_sma_minPos
6. pcm_zcr_sma_de_lpc1
7. pcm_zcr_sma_percentile99.0
8. pcm_zcr_sma_de_pctlrange0-1
9. pcm_zcr_sma_quartile2
10. pcm_zcr_sma_range


In [ ]:
smFS_model.select_features_binary(max_features=10, archs='UberDuck')


Selected features:.

1. mfcc_sma[5]_lpc2
2. mfcc_sma[13]_lpgain
3. mfcc_sma_de[13]_meanFallingSlope
4. mfcc_sma_de[13]_meanRisingSlope
5. mfcc_sma_de[13]_lpgain
6. mfcc_sma[14]_peakMeanAbs
7. mfcc_sma_de[8]_lpgain
8. mfcc_sma[14]_rqmean
9. mfcc_sma[13]_meanFallingSlope
10. mfcc_sma[14]_quartile2


In [ ]:
smFS_model.select_features_multiclass(max_features=10, archs=['ElevenLabs', 'UberDuck'], return_features=True)


Selected features:.

1. mfcc_sma[5]_lpc2
2. voicingFinalUnclipped_sma_rqmean
3. jitterDDP_sma_quartile1
4. audspec_lengthL1norm_sma_de_stddev
5. audspec_lengthL1norm_sma_de_peakMeanMeanDist
6. jitterDDP_sma_percentile1.0
7. pcm_fftMag_spectralSlope_sma_percentile99.0
8. pcm_zcr_sma_de_peakMeanMeanDist
9. pcm_zcr_sma_de_rqmean
10. mfcc_sma[11]_lpc2


['mfcc_sma[5]_lpc2',
 'voicingFinalUnclipped_sma_rqmean',
 'jitterDDP_sma_quartile1',
 'audspec_lengthL1norm_sma_de_stddev',
 'audspec_lengthL1norm_sma_de_peakMeanMeanDist',
 'jitterDDP_sma_percentile1.0',
 'pcm_fftMag_spectralSlope_sma_percentile99.0',
 'pcm_zcr_sma_de_peakMeanMeanDist',
 'pcm_zcr_sma_de_rqmean',
 'mfcc_sma[11]_lpc2']

### Univariate Feature Selection:

In [ ]:
smFS_univariate = smileUnivariateFeatureSelector(df=fs_df)

Initializing data...
Standardizing features...
smileUnivariateFeatureSelector object initialized.



In [ ]:
smFS_univariate.select_features(score_func=fs.mutual_info_classif, mode='k_best',
                                num_features=25, archs='all_archs')


Selected features:.

1. pcm_zcr_sma_quartile1
2. mfcc_sma[13]_lpgain
3. pcm_fftMag_spectralFlux_sma_minPos
4. mfcc_sma[6]_lpc0
5. pcm_zcr_sma_percentile99.0
6. pcm_zcr_sma_de_range
7. audspec_lengthL1norm_sma_peakMeanAbs
8. pcm_RMSenergy_sma_de_peakMeanRel
9. audspec_lengthL1norm_sma_de_stddev
10. audspec_lengthL1norm_sma_de_rqmean
11. pcm_zcr_sma_pctlrange0-1
12. audspec_lengthL1norm_sma_quartile3
13. pcm_zcr_sma_de_pctlrange0-1
14. pcm_fftMag_spectralSlope_sma_percentile99.0
15. pcm_fftMag_spectralHarmonicity_sma_de_peakMeanRel
16. audspec_lengthL1norm_sma_de_posamean
17. audspec_lengthL1norm_sma_rqmean
18. pcm_zcr_sma_quartile2
19. pcm_zcr_sma_percentile1.0
20. pcm_fftMag_spectralSlope_sma_maxPos
21. pcm_zcr_sma_range
22. audspec_lengthL1norm_sma_amean
23. voicingFinalUnclipped_sma_percentile1.0
24. mfcc_sma_de[12]_lpgain
25. pcm_zcr_sma_de_percentile1.0


In [ ]:
smFS_univariate.select_features(score_func=fs.mutual_info_classif, mode='k_best',
                                num_features=25, archs=['ElevenLabs', 'UberDuck'])


Selected features:.

1. pcm_zcr_sma_quartile1
2. pcm_fftMag_spectralFlux_sma_minPos
3. pcm_zcr_sma_percentile99.0
4. pcm_zcr_sma_de_range
5. audspec_lengthL1norm_sma_peakMeanAbs
6. mfcc_sma[14]_rqmean
7. pcm_RMSenergy_sma_de_peakMeanRel
8. audspec_lengthL1norm_sma_de_stddev
9. pcm_fftMag_spectralKurtosis_sma_de_quartile3
10. pcm_fftMag_spectralKurtosis_sma_de_iqr2-3
11. audspec_lengthL1norm_sma_de_rqmean
12. pcm_zcr_sma_pctlrange0-1
13. pcm_zcr_sma_de_pctlrange0-1
14. pcm_fftMag_spectralSlope_sma_percentile99.0
15. audspec_lengthL1norm_sma_rqmean
16. audspec_lengthL1norm_sma_minPos
17. pcm_zcr_sma_quartile2
18. pcm_zcr_sma_percentile1.0
19. pcm_fftMag_spectralSlope_sma_maxPos
20. mfcc_sma[14]_quartile1
21. pcm_zcr_sma_range
22. audspec_lengthL1norm_sma_amean
23. pcm_zcr_sma_de_percentile99.0
24. pcm_RMSenergy_sma_minPos
25. voicingFinalUnclipped_sma_percentile1.0


In [ ]:
smFS_univariate.select_features(score_func=fs.mutual_info_classif, mode='k_best',
                                num_features=25, archs='ElevenLabs')


Selected features:.

1. pcm_zcr_sma_quartile1
2. pcm_fftMag_spectralFlux_sma_minPos
3. pcm_zcr_sma_peakMeanAbs
4. pcm_zcr_sma_percentile99.0
5. pcm_zcr_sma_de_range
6. pcm_fftMag_spectralHarmonicity_sma_minPos
7. pcm_RMSenergy_sma_de_peakMeanRel
8. pcm_zcr_sma_pctlrange0-1
9. pcm_fftMag_fband250-650_sma_minPos
10. pcm_zcr_sma_de_pctlrange0-1
11. pcm_fftMag_spectralSlope_sma_percentile99.0
12. pcm_zcr_sma_peakRangeAbs
13. pcm_zcr_sma_de_rqmean
14. pcm_zcr_sma_stddev
15. pcm_zcr_sma_amean
16. audspec_lengthL1norm_sma_minPos
17. pcm_zcr_sma_quartile2
18. pcm_zcr_sma_percentile1.0
19. pcm_fftMag_spectralSlope_sma_maxPos
20. pcm_zcr_sma_range
21. pcm_zcr_sma_rqmean
22. pcm_zcr_sma_de_percentile99.0
23. pcm_RMSenergy_sma_minPos
24. pcm_zcr_sma_de_peakRangeAbs
25. pcm_zcr_sma_de_percentile1.0


In [ ]:
smFS_univariate.select_features(score_func=fs.mutual_info_classif, mode='k_best',
                                num_features=25, archs='UberDuck')


Selected features:.

1. mfcc_sma[13]_lpgain
2. mfcc_sma_de[9]_meanRisingSlope
3. mfcc_sma_de[9]_stddev
4. mfcc_sma_de[13]_iqr1-3
5. mfcc_sma[14]_quartile2
6. mfcc_sma[5]_lpc2
7. mfcc_sma[14]_meanRisingSlope
8. mfcc_sma_de[14]_meanFallingSlope
9. mfcc_sma[14]_quartile3
10. mfcc_sma[3]_lpc2
11. mfcc_sma[10]_meanPeakDist
12. mfcc_sma[14]_amean
13. mfcc_sma_de[13]_meanRisingSlope
14. mfcc_sma_de[13]_lpgain
15. mfcc_sma[14]_peakMeanAbs
16. mfcc_sma[14]_peakMeanRel
17. mfcc_sma_de[9]_meanFallingSlope
18. mfcc_sma[11]_lpc2
19. mfcc_sma_de[13]_meanFallingSlope
20. mfcc_sma[8]_lpgain
21. mfcc_sma[9]_lpgain
22. mfcc_sma_de[9]_iqr1-3
23. mfcc_sma_de[8]_lpgain
24. mfcc_sma[12]_lpgain
25. mfcc_sma_de[9]_lpgain


# Testing OpenSmile Classes

Draft 1 of pipeline for creating a df, loading the generated openSMILE features, and running feature selection.

In [ ]:
import pandas as pd
import numpy as np
import sys
import os
import json
from tqdm import tqdm
sys.path.append("/home/ubuntu/MultiModalDeepFake")
from packages.LJDataLoader import LJDataLoader
from packages.SmileFeatureLoader import smileFeatureLoader
from packages.SmileFeatureSelector import smileFeatureSelectorBruteForce, smileFeatureSelectFromModel, smileUnivariateFeatureSelector, process_data_for_all_archs
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import sklearn.feature_selection as fs
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, confusion_matrix, classification_report

## Part 1: Creating the dataframe with the train-dev-test split

In [ ]:
#load LJ 16khz data
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_Original.csv'
loader = LJDataLoader(data_path=file_path)

In [ ]:
#split data into train, dev, test
loader.splitData()

13057
7834 10445


In [ ]:
source_architectures = ['Full_Band_MelGan', 'HifiGan', 'MelGan', 'MelGanLarge', 'Multi_Band_MelGan', 'Parallel_WaveGan', 'Waveglow']
new_col_name = 'RandWaveFake'

In [ ]:
loader.selectRandomArchitecture(target_col=new_col_name, source_cols=source_architectures)

In [ ]:
loader.metadata.columns

Index(['index', 'Unnamed: 0', 'id', 'Main_ID', 'Secondary_ID', 'Real',
       'Full_Band_MelGan', 'HifiGan', 'MelGan', 'MelGanLarge',
       'Multi_Band_MelGan', 'Parallel_WaveGan', 'Waveglow', 'ElevenLabs',
       'UberDuck', 'transcript_1', 'transcript_2', 'type', 'RandWaveFake'],
      dtype='object')

In [ ]:
#generate the dataframe
#single_id entry=True breaks
df = loader.generateFinalDataFrame(real_col='Real', fake_cols=['ElevenLabs','UberDuck','RandWaveFake'])

In [ ]:
df.shape

(52228, 6)

In [ ]:
df.head()

,type,id,architecture,path,label,multiclass_label
0,train,LJ013-0125,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
1,train,LJ042-0084,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
2,train,LJ047-0032,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
3,train,LJ044-0063,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
4,train,LJ016-0067,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0


In [ ]:
df['architecture'].value_counts()

Real            13057
ElevenLabs      13057
UberDuck        13057
RandWaveFake    13057
Name: architecture, dtype: int64

#### Temporary solution for all_archs:

This external function could be integrated into data loader class

In [ ]:
fake_cols=['ElevenLabs','UberDuck','RandWaveFake']

arch_filter = process_data_for_all_archs(df, fake_cols)

In [ ]:
for key,value in arch_filter.items():
    print(key, len(value))

ElevenLabs 4352
UberDuck 4352
RandWaveFake 4353


In [ ]:
df2 = pd.DataFrame()
for key, value in arch_filter.items():
    df_ = df[(df['architecture'].isin(['Real', key])) & (df['id'].isin(value))].copy()
    df_.loc[:, 'arch_filter'] = [key] * df_.shape[0]
    df2 = pd.concat((df2, df_))

In [ ]:
df2.architecture.value_counts()

Real            13057
RandWaveFake     4353
ElevenLabs       4352
UberDuck         4352
Name: architecture, dtype: int64

In [ ]:
df2.arch_filter.value_counts()

RandWaveFake    8706
ElevenLabs      8704
UberDuck        8704
Name: arch_filter, dtype: int64

In [ ]:
merge_df = df2[['path','arch_filter']]

In [ ]:
merge_df.shape, df2.shape

((26114, 2), (26114, 7))

In [ ]:
final_df = pd.merge(df, merge_df, on='path', how='left')

In [ ]:
final_df.arch_filter.value_counts(dropna=False)

NaN             26114
RandWaveFake     8706
UberDuck         8704
ElevenLabs       8704
Name: arch_filter, dtype: int64

In [ ]:
df.shape, final_df.shape

((52228, 6), (52228, 7))

## Side: Creating/Updating File Mappings JSON:

This JSON has the path to the collection of wavfiles as the key and the corresponding smile features as values. 

For now, each time a new set of wav file are generated for a new provider/architecture - they should be stored in a separate unique folderpath. The `SmileFeatureGenerator` class will convert these to the csv file that can then be used to create a new JSON entry. 

In [ ]:
file_mapping = dict()

#biden real
file_mapping['/home/ubuntu/data/world-leaders-dataset/WavFiles/biden_wav_audio/44KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-world-leaders-dataset-WavFiles-biden_wav_audio-44KHz.csv'
file_mapping['/home/ubuntu/data/world-leaders-dataset/WavFiles/biden_wav_audio/16KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-world-leaders-dataset-WavFiles-biden_wav_audio-16KHz.csv'

#biden eleven labs
file_mapping['/home/ubuntu/data/world-leaders-dataset/11LabsDeepFakes/Biden/44Khz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-world-leaders-dataset-11LabsDeepFakes-Biden-44Khz.csv'
file_mapping['/home/ubuntu/data/world-leaders-dataset/11LabsDeepFakes/Biden/16Khz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-world-leaders-dataset-11LabsDeepFakes-Biden-16Khz.csv'

#biden in the wild
file_mapping['/home/ubuntu/data/InTheWildFakes/Biden/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-InTheWildFakes-Biden-Original.csv'
file_mapping['/home/ubuntu/data/InTheWildFakes/Biden/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-InTheWildFakes-Biden-16000KHz.csv'

#ljspeech real
file_mapping['/home/ubuntu/data/wavefake_data/LJSpeech_1.1/wavs/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-LJSpeech_1.1-wavs-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/LJSpeech_1.1/wavs/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-LJSpeech_1.1-wavs-16000KHz.csv'

#ljspeech eleven labs
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_elevenlabs/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_elevenlabs-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_elevenlabs/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_elevenlabs-16000KHz.csv'

#ljspeech uberduck
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_uberduck/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-06-wavefake_data-generated_audio-ljspeech_uberduck-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_uberduck/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-06-wavefake_data-generated_audio-ljspeech_uberduck-16000KHz.csv'

#ljspeech melgan
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_melgan/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_melgan-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_melgan/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_melgan-16000KHz.csv'

#ljspeech melgan large
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_melgan_large/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_melgan_large-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_melgan_large/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_melgan_large-16000KHz.csv'

#ljspeech multi band melgan
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_multi_band_melgan/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_multi_band_melgan-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_multi_band_melgan/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_multi_band_melgan-16000KHz.csv'

#ljspeech full band melgan
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_full_band_melgan/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_full_band_melgan-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_full_band_melgan/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_full_band_melgan-16000KHz.csv'

#ljspeech hifiGAN
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_hifiGAN/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_hifiGAN-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_hifiGAN/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_hifiGAN-16000KHz.csv'

#ljspeech parallel wavegan
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_parallel_wavegan/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_parallel_wavegan-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_parallel_wavegan/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_parallel_wavegan-16000KHz.csv'

#ljspeech waveglow
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_waveglow/Original'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_waveglow-Original.csv'
file_mapping['/home/ubuntu/data/wavefake_data/generated_audio/ljspeech_waveglow/16000KHz'] = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/OSF-2023-04-01-wavefake_data-generated_audio-ljspeech_waveglow-16000KHz.csv'

#file_mapping

In [ ]:
cellEnabled=0

if cellEnabled:

    file_mappings = '/home/ubuntu/data/OpenSmile/04-2023-OSF-feature-csvs/' + 'file_mappings.json' 

    with open(file_mappings, "w") as json_file:
        json.dump(file_mapping, json_file, indent=4)

## Part 2: Loading the openSMILE features

The openSMILE features are generated and stored in multiple `csv` files here - `data/OpenSmile/04-2023-OSF-feature-csvs`. This folder also stores the JSON file

In [ ]:
final_df.shape

(52228, 7)

In [ ]:
#pass the test df and the smileFeatureLoader object does the merging
smFeatLoader = smileFeatureLoader(final_df)

100%|██████████| 10/10 [02:15<00:00, 13.54s/it]


In [ ]:
#get the df for feature selection
fs_df = smFeatLoader.return_merged_df()

In [ ]:
fs_df.shape

(52226, 6381)

In [ ]:
df.shape

(52228, 6)

In [ ]:
fs_df.head()

,type,id,architecture,path,label,multiclass_label,arch_filter,duration(seconds),audspec_lengthL1norm_sma_range,audspec_lengthL1norm_sma_maxPos,...,mfcc_sma_de[14]_peakRangeAbs,mfcc_sma_de[14]_peakRangeRel,mfcc_sma_de[14]_peakMeanAbs,mfcc_sma_de[14]_peakMeanMeanDist,mfcc_sma_de[14]_peakMeanRel,mfcc_sma_de[14]_minRangeRel,mfcc_sma_de[14]_meanRisingSlope,mfcc_sma_de[14]_stddevRisingSlope,mfcc_sma_de[14]_meanFallingSlope,mfcc_sma_de[14]_stddevFallingSlope
0,train,LJ013-0125,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,UberDuck,7.600,2.150976,0.856192,...,6.666750,0.561728,2.251509,2.245225,19.983110,0.502100,104.51715,42.642437,102.34870,44.233257
1,train,LJ042-0084,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,RandWaveFake,8.517,2.152849,0.801663,...,8.883474,0.466906,3.512975,3.513055,-20.000000,0.595703,138.78543,64.106000,135.07686,69.522630
2,train,LJ047-0032,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,ElevenLabs,8.203,1.993151,0.070284,...,9.228073,0.445313,3.162120,3.158883,19.999882,0.725314,130.25446,67.431730,124.56447,68.578705
3,train,LJ044-0063,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,RandWaveFake,3.025,3.101600,0.054422,...,7.837853,0.519312,2.595290,2.634245,-18.663334,0.602941,114.82919,67.769290,113.65942,59.367180
4,train,LJ016-0067,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0,RandWaveFake,10.049,2.455078,0.045226,...,9.766602,0.581217,2.960240,2.954486,19.996292,0.609338,137.75078,60.657215,129.61658,60.697903


In [ ]:
#get the columns that are not the 6373 features as metadata
metadata = list(fs_df.columns[:7])
metadata

['type',
 'id',
 'architecture',
 'path',
 'label',
 'multiclass_label',
 'arch_filter']

## Part 3: Running Brute Force Feature Selection

In [ ]:
fs_df.shape

(52226, 6381)

In [ ]:
#initialize the object
smFS_bf = smileFeatureSelectorBruteForce(df=fs_df)

Initializing data...
Standardizing features...
smileFeatureSelectorBruteForce object initialized.



In [ ]:
smFS_bf.generate_data()

Generating brute force feature selection data...


100%|██████████| 6373/6373 [04:23<00:00, 24.14it/s]


Brute force feature selection data generated.
All data stored in self.bffs_data.



In [ ]:
smFS_bf.bffs_data.sort_values(by='ElevenLabs', ascending=False)

,features,ElevenLabs,UberDuck,RandWaveFake,average_binary,all_archs,multiclass
96,pcm_zcr_sma_quartile1,0.998468,0.600153,0.500574,0.699732,0.662007,0.548640
93,pcm_zcr_sma_range,0.995596,0.580429,0.513022,0.696349,0.653198,0.529395
103,pcm_zcr_sma_percentile99.0,0.995021,0.541172,0.518767,0.684987,0.641708,0.517714
104,pcm_zcr_sma_pctlrange0-1,0.994255,0.533321,0.521256,0.682944,0.640176,0.513118
227,pcm_zcr_sma_de_percentile99.0,0.993489,0.556683,0.519341,0.689838,0.622941,0.534374
...,...,...,...,...,...,...,...
2562,audSpec_Rfilt_sma_de[19]_upleveltime50,0.457488,0.513596,0.501340,0.490808,0.476446,0.234872
4862,pcm_fftMag_fband1000-4000_sma_qregc1,0.457296,0.548449,0.496936,0.500894,0.528342,0.251628
175,audspecRasta_lengthL1norm_sma_de_upleveltime50,0.454998,0.543278,0.504404,0.500894,0.497319,0.270203
2493,audSpec_Rfilt_sma_de[17]_skewness,0.453083,0.557641,0.500574,0.503766,0.496170,0.280065


In [ ]:
smFS_bf.FS_from_col(num_features=15, sort_col='ElevenLabs')


Top 15 features selected from ElevenLabs architecture.

1. pcm_zcr_sma_quartile1
2. pcm_zcr_sma_range
3. pcm_zcr_sma_percentile99.0
4. pcm_zcr_sma_pctlrange0-1
5. pcm_zcr_sma_de_percentile99.0
6. pcm_zcr_sma_de_pctlrange0-1
7. pcm_zcr_sma_quartile2
8. pcm_zcr_sma_de_range
9. pcm_zcr_sma_de_percentile1.0
10. pcm_zcr_sma_percentile1.0
11. pcm_fftMag_spectralSlope_sma_percentile99.0
12. audspec_lengthL1norm_sma_de_peakMeanRel
13. pcm_zcr_sma_stddev
14. pcm_zcr_sma_rqmean
15. pcm_zcr_sma_peakMeanAbs


In [ ]:
smFS_bf.FS_top_n_from_all_archs(n=15)


Top features selected from all architectures.

1. pcm_zcr_sma_quartile1
2. jitterLocal_sma_flatness
3. mfcc_sma[13]_lpgain
4. mfcc_sma_de[9]_meanRisingSlope
5. jitterDDP_sma_quartile2
6. pcm_zcr_sma_peakMeanAbs
7. pcm_zcr_sma_percentile99.0
8. pcm_zcr_sma_de_range
9. mfcc_sma_de[14]_lpgain
10. shimmerLocal_sma_quartile1
11. mfcc_sma[14]_quartile2
12. mfcc_sma[5]_lpc2
13. jitterDDP_sma_flatness
14. jitterDDP_sma_percentile1.0
15. pcm_zcr_sma_pctlrange0-1
16. shimmerLocal_sma_percentile1.0
17. pcm_zcr_sma_de_pctlrange0-1
18. pcm_fftMag_spectralSlope_sma_percentile99.0
19. mfcc_sma_de[11]_lpgain
20. mfcc_sma_de[14]_meanFallingSlope
21. mfcc_sma[14]_quartile3
22. jitterLocal_sma_quartile1
23. pcm_zcr_sma_stddev
24. mfcc_sma[14]_amean
25. mfcc_sma_de[13]_meanRisingSlope
26. jitterLocal_sma_percentile1.0
27. mfcc_sma_de[13]_lpgain
28. mfcc_sma[14]_peakMeanAbs
29. pcm_zcr_sma_quartile2
30. pcm_zcr_sma_percentile1.0
31. pcm_zcr_sma_rqmean
32. pcm_zcr_sma_range
33. mfcc_sma[11]_lpc2
34. audspe

#### Running multiclass prediction:

In [ ]:
features = smFS_bf.FS_top_n_from_all_archs(n=15, return_features=True, print_features=False)

In [ ]:
metadata=['type', 'id', 'architecture', 'path', 'label', 'multiclass_label', 'duration(seconds)']
ml_data = fs_df[fs_df.columns.intersection(metadata + features)]

In [ ]:
model = LogisticRegression(max_iter=1000000, n_jobs=-1, solver='saga')

In [ ]:
train_df = ml_data[ml_data['type'] == 'train'].copy()
test_df = ml_data[ml_data['type'] == 'test'].copy()

In [ ]:
X_train = train_df[features]
X_test = test_df[features]

y_train = train_df['multiclass_label']
y_test = test_df['multiclass_label']

model.fit(X_train, y_train)
y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)

tra_acc = accuracy_score(y_train, y_hat_train)
tst_acc = accuracy_score(y_test, y_hat_test)

tra_acc, tst_acc

(0.8895484282750917, 0.8893462237963051)

## Part 4: Running Select From Model

In [ ]:
smFS_model = smileFeatureSelectFromModel(fs_df)

Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.



In [ ]:
smFS_model.select_features_binary()


Selected features:.

1. mfcc_sma[13]_lpgain
2. mfcc_sma[8]_lpgain
3. jitterDDP_sma_quartile1
4. mfcc_sma[9]_lpgain
5. jitterDDP_sma_percentile1.0
6. pcm_zcr_sma_percentile99.0
7. mfcc_sma_de[13]_lpgain
8. mfcc_sma[12]_lpgain
9. mfcc_sma_de[9]_lpgain
10. mfcc_sma[12]_meanPeakDist


In [ ]:
smFS_model.select_features_binary(max_features=10, archs=['ElevenLabs', 'UberDuck'])


Selected features:.

1. mfcc_sma[5]_lpc2
2. mfcc_sma[13]_lpgain
3. audspec_lengthL1norm_sma_rqmean
4. mfcc_sma[9]_lpgain
5. pcm_zcr_sma_percentile99.0
6. mfcc_sma_de[13]_lpgain
7. mfcc_sma[14]_peakMeanAbs
8. pcm_zcr_sma_peakRangeAbs
9. mfcc_sma_de[9]_lpgain
10. mfcc_sma[14]_quartile2


In [ ]:
smFS_model.select_features_binary(max_features=10, archs='ElevenLabs')


Selected features:.

1. pcm_zcr_sma_quartile1
2. pcm_zcr_sma_stddev
3. pcm_zcr_sma_rqmean
4. voicingFinalUnclipped_sma_flatness
5. audspec_lengthL1norm_sma_minPos
6. pcm_zcr_sma_de_lpc1
7. pcm_zcr_sma_percentile99.0
8. pcm_zcr_sma_de_pctlrange0-1
9. pcm_zcr_sma_quartile2
10. pcm_zcr_sma_range


In [ ]:
smFS_model.select_features_binary(max_features=10, archs='UberDuck')


Selected features:.

1. mfcc_sma[5]_lpc2
2. mfcc_sma[13]_lpgain
3. mfcc_sma_de[13]_meanFallingSlope
4. mfcc_sma_de[13]_meanRisingSlope
5. mfcc_sma_de[13]_lpgain
6. mfcc_sma[14]_peakMeanAbs
7. mfcc_sma_de[8]_lpgain
8. mfcc_sma[14]_rqmean
9. mfcc_sma[13]_meanFallingSlope
10. mfcc_sma[14]_quartile2


In [ ]:
smFS_model.select_features_multiclass(max_features=10, archs=['ElevenLabs', 'UberDuck'], return_features=True)


Selected features:.

1. mfcc_sma[5]_lpc2
2. voicingFinalUnclipped_sma_rqmean
3. jitterDDP_sma_quartile1
4. audspec_lengthL1norm_sma_de_stddev
5. audspec_lengthL1norm_sma_de_peakMeanMeanDist
6. jitterDDP_sma_percentile1.0
7. pcm_fftMag_spectralSlope_sma_percentile99.0
8. pcm_zcr_sma_de_peakMeanMeanDist
9. pcm_zcr_sma_de_rqmean
10. mfcc_sma[11]_lpc2


['mfcc_sma[5]_lpc2',
 'voicingFinalUnclipped_sma_rqmean',
 'jitterDDP_sma_quartile1',
 'audspec_lengthL1norm_sma_de_stddev',
 'audspec_lengthL1norm_sma_de_peakMeanMeanDist',
 'jitterDDP_sma_percentile1.0',
 'pcm_fftMag_spectralSlope_sma_percentile99.0',
 'pcm_zcr_sma_de_peakMeanMeanDist',
 'pcm_zcr_sma_de_rqmean',
 'mfcc_sma[11]_lpc2']

### Univariate Feature Selection:

In [ ]:
smFS_univariate = smileUnivariateFeatureSelector(df=fs_df)

Initializing data...
Standardizing features...
smileUnivariateFeatureSelector object initialized.



In [ ]:
smFS_univariate.select_features(score_func=fs.mutual_info_classif, mode='k_best',
                                num_features=25, archs='all_archs')


Selected features:.

1. pcm_zcr_sma_quartile1
2. mfcc_sma[13]_lpgain
3. pcm_fftMag_spectralFlux_sma_minPos
4. mfcc_sma[6]_lpc0
5. pcm_zcr_sma_percentile99.0
6. pcm_zcr_sma_de_range
7. audspec_lengthL1norm_sma_peakMeanAbs
8. pcm_RMSenergy_sma_de_peakMeanRel
9. audspec_lengthL1norm_sma_de_stddev
10. audspec_lengthL1norm_sma_de_rqmean
11. pcm_zcr_sma_pctlrange0-1
12. audspec_lengthL1norm_sma_quartile3
13. pcm_zcr_sma_de_pctlrange0-1
14. pcm_fftMag_spectralSlope_sma_percentile99.0
15. pcm_fftMag_spectralHarmonicity_sma_de_peakMeanRel
16. audspec_lengthL1norm_sma_de_posamean
17. audspec_lengthL1norm_sma_rqmean
18. pcm_zcr_sma_quartile2
19. pcm_zcr_sma_percentile1.0
20. pcm_fftMag_spectralSlope_sma_maxPos
21. pcm_zcr_sma_range
22. audspec_lengthL1norm_sma_amean
23. voicingFinalUnclipped_sma_percentile1.0
24. mfcc_sma_de[12]_lpgain
25. pcm_zcr_sma_de_percentile1.0


In [ ]:
smFS_univariate.select_features(score_func=fs.mutual_info_classif, mode='k_best',
                                num_features=25, archs=['ElevenLabs', 'UberDuck'])


Selected features:.

1. pcm_zcr_sma_quartile1
2. pcm_fftMag_spectralFlux_sma_minPos
3. pcm_zcr_sma_percentile99.0
4. pcm_zcr_sma_de_range
5. audspec_lengthL1norm_sma_peakMeanAbs
6. mfcc_sma[14]_rqmean
7. pcm_RMSenergy_sma_de_peakMeanRel
8. audspec_lengthL1norm_sma_de_stddev
9. pcm_fftMag_spectralKurtosis_sma_de_quartile3
10. pcm_fftMag_spectralKurtosis_sma_de_iqr2-3
11. audspec_lengthL1norm_sma_de_rqmean
12. pcm_zcr_sma_pctlrange0-1
13. pcm_zcr_sma_de_pctlrange0-1
14. pcm_fftMag_spectralSlope_sma_percentile99.0
15. audspec_lengthL1norm_sma_rqmean
16. audspec_lengthL1norm_sma_minPos
17. pcm_zcr_sma_quartile2
18. pcm_zcr_sma_percentile1.0
19. pcm_fftMag_spectralSlope_sma_maxPos
20. mfcc_sma[14]_quartile1
21. pcm_zcr_sma_range
22. audspec_lengthL1norm_sma_amean
23. pcm_zcr_sma_de_percentile99.0
24. pcm_RMSenergy_sma_minPos
25. voicingFinalUnclipped_sma_percentile1.0


In [ ]:
smFS_univariate.select_features(score_func=fs.mutual_info_classif, mode='k_best',
                                num_features=25, archs='ElevenLabs')


Selected features:.

1. pcm_zcr_sma_quartile1
2. pcm_fftMag_spectralFlux_sma_minPos
3. pcm_zcr_sma_peakMeanAbs
4. pcm_zcr_sma_percentile99.0
5. pcm_zcr_sma_de_range
6. pcm_fftMag_spectralHarmonicity_sma_minPos
7. pcm_RMSenergy_sma_de_peakMeanRel
8. pcm_zcr_sma_pctlrange0-1
9. pcm_fftMag_fband250-650_sma_minPos
10. pcm_zcr_sma_de_pctlrange0-1
11. pcm_fftMag_spectralSlope_sma_percentile99.0
12. pcm_zcr_sma_peakRangeAbs
13. pcm_zcr_sma_de_rqmean
14. pcm_zcr_sma_stddev
15. pcm_zcr_sma_amean
16. audspec_lengthL1norm_sma_minPos
17. pcm_zcr_sma_quartile2
18. pcm_zcr_sma_percentile1.0
19. pcm_fftMag_spectralSlope_sma_maxPos
20. pcm_zcr_sma_range
21. pcm_zcr_sma_rqmean
22. pcm_zcr_sma_de_percentile99.0
23. pcm_RMSenergy_sma_minPos
24. pcm_zcr_sma_de_peakRangeAbs
25. pcm_zcr_sma_de_percentile1.0


In [ ]:
smFS_univariate.select_features(score_func=fs.mutual_info_classif, mode='k_best',
                                num_features=25, archs='UberDuck')


Selected features:.

1. mfcc_sma[13]_lpgain
2. mfcc_sma_de[9]_meanRisingSlope
3. mfcc_sma_de[9]_stddev
4. mfcc_sma_de[13]_iqr1-3
5. mfcc_sma[14]_quartile2
6. mfcc_sma[5]_lpc2
7. mfcc_sma[14]_meanRisingSlope
8. mfcc_sma_de[14]_meanFallingSlope
9. mfcc_sma[14]_quartile3
10. mfcc_sma[3]_lpc2
11. mfcc_sma[10]_meanPeakDist
12. mfcc_sma[14]_amean
13. mfcc_sma_de[13]_meanRisingSlope
14. mfcc_sma_de[13]_lpgain
15. mfcc_sma[14]_peakMeanAbs
16. mfcc_sma[14]_peakMeanRel
17. mfcc_sma_de[9]_meanFallingSlope
18. mfcc_sma[11]_lpc2
19. mfcc_sma_de[13]_meanFallingSlope
20. mfcc_sma[8]_lpgain
21. mfcc_sma[9]_lpgain
22. mfcc_sma_de[9]_iqr1-3
23. mfcc_sma_de[8]_lpgain
24. mfcc_sma[12]_lpgain
25. mfcc_sma_de[9]_lpgain
